In [ ]:
import jax
import jax.numpy as jnp

global_list = []


def log2(x):
    global_list.append(x)  # not in jaxpr, not pure function
    ln_x = jnp.log(x)
    ln_2 = jnp.log(2.0)
    return ln_x / ln_2


print(jax.make_jaxpr(log2)(3.0))

{ lambda ; a:f32[]. let
    b:f32[] = log a
    c:f32[] = log 2.0
    d:f32[] = div b c
  in (d,) }


Impure functions are dangerous because under JAX transformations they are likely not to behave as intended; they might fail silently, or produce surprising downstream errors like leaked Tracers. Moreover, JAX often can’t detect when side effects are present. (If you want debug printing, use `jax.debug.print()`. To express general side-effects at the cost of performance, see `jax.experimental.io_callback()`. To check for tracer leaks at the cost of performance, use with `jax.check_tracer_leaks()`).

In [5]:
def log2_with_print(x):
    print("printed x:", x)
    ln_x = jnp.log(x)
    ln_2 = jnp.log(2.0)
    return ln_x / ln_2


print(jax.make_jaxpr(log2_with_print)(3.0))

printed x: Traced<ShapedArray(float32[], weak_type=True)>with<DynamicJaxprTrace(level=1/0)>
{ lambda ; a:f32[]. let
    b:f32[] = log a
    c:f32[] = log 2.0
    d:f32[] = div b c
  in (d,) }


In [ ]:
# A key thing to understand is that a jaxpr captures the function as executed on the parameters given to it. For example, if we have a Python conditional, the jaxpr will only know about the branch we take:
def log2_if_rank_2(x):
    if x.ndim == 2:
        ln_x = jnp.log(x)
        ln_2 = jnp.log(2.0)
        return ln_x / ln_2
    else:
        return x


print(jax.make_jaxpr(log2_if_rank_2)(jax.numpy.array([1, 2, 3])))

{ lambda ; a:i32[3]. let  in (a,) }


In [7]:
import jax
import jax.numpy as jnp


def selu(x, alpha=1.67, lambda_=1.05):
    return lambda_ * jnp.where(x > 0, x, alpha * jnp.exp(x) - alpha)


x = jnp.arange(1000000)
%timeit selu(x).block_until_ready()

selu_jit = jax.jit(selu)

# Pre-compile the function before timing...
selu_jit(x).block_until_ready()

%timeit selu_jit(x).block_until_ready()

1.04 ms ± 61.2 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
750 μs ± 89.4 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [8]:
# Condition on value of x.


def f(x):
    if x > 0:
        return x
    else:
        return 2 * x


jax.jit(f)(10)  # Raises an error

TracerBoolConversionError: Attempted boolean conversion of traced array with shape bool[].
The error occurred while tracing the function f at /tmp/ipykernel_14187/617957807.py:4 for jit. This concrete value was not available in Python because it depends on the value of the argument x.
See https://jax.readthedocs.io/en/latest/errors.html#jax.errors.TracerBoolConversionError

In [9]:
# While loop conditioned on x and n.


def g(x, n):
    i = 0
    while i < n:
        i += 1
    return x + i


jax.jit(g)(10, 20)  # Raises an error

TracerBoolConversionError: Attempted boolean conversion of traced array with shape bool[].
The error occurred while tracing the function g at /tmp/ipykernel_14187/3605073761.py:4 for jit. This concrete value was not available in Python because it depends on the value of the argument n.
See https://jax.readthedocs.io/en/latest/errors.html#jax.errors.TracerBoolConversionError

Traced values within JIT, like x and n here, can only affect control flow via their static attributes: such as `shape` or `dtype`, and not via their values.

In [10]:
# While loop conditioned on x and n with a jitted body.


@jax.jit
def loop_body(prev_i):
    return prev_i + 1


def g_inner_jitted(x, n):
    i = 0
    while i < n:
        i = loop_body(i)
    return x + i


g_inner_jitted(10, 20)

Array(30, dtype=int32, weak_type=True)

The cost of this is that the resulting jaxpr and compiled artifact depends on the particular value passed, and so JAX will have to re-compile the function for every new value of the specified static input. It is only a good strategy if the function is guaranteed to see a limited set of static values.

In [11]:
f_jit_correct = jax.jit(f, static_argnums=0)
print(f_jit_correct(10))

10


In [ ]:
g_jit_correct = jax.jit(g, static_argnames=["n"])
print(g_jit_correct(10, 20))

30


In [ ]:
from functools import partial


@partial(jax.jit, static_argnames=["n"])
def g_jit_decorated(x, n):
    i = 0
    while i < n:
        i += 1
    return x + i


print(g_jit_decorated(10, 20))

30


In [14]:
from functools import partial


def unjitted_loop_body(prev_i):
    return prev_i + 1


def g_inner_jitted_partial(x, n):
    i = 0
    while i < n:
        # Don't do this! each time the partial returns
        # a function with different hash
        i = jax.jit(partial(unjitted_loop_body))(i)
    return x + i


def g_inner_jitted_lambda(x, n):
    i = 0
    while i < n:
        # Don't do this!, lambda will also return
        # a function with a different hash
        i = jax.jit(lambda x: unjitted_loop_body(x))(i)
    return x + i


def g_inner_jitted_normal(x, n):
    i = 0
    while i < n:
        # this is OK, since JAX can find the
        # cached, compiled function
        i = jax.jit(unjitted_loop_body)(i)
    return x + i


print("jit called in a loop with partials:")
%timeit g_inner_jitted_partial(10, 20).block_until_ready()

print("jit called in a loop with lambdas:")
%timeit g_inner_jitted_lambda(10, 20).block_until_ready()

print("jit called in a loop with caching:")
%timeit g_inner_jitted_normal(10, 20).block_until_ready()

jit called in a loop with partials:
251 ms ± 5.48 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
jit called in a loop with lambdas:
241 ms ± 4.32 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
jit called in a loop with caching:
8.22 ms ± 154 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
